In [1]:
# Break in a little-bit
# We want the Java-level operation here, that's all
# !pip install langgraph
from src.utils import *

#### Progress:
* PlayWright only support Async call which complicate things a little bit (but fine it's just another wrapper layer)
* Bounding Boxes & Screenshot function is taken care of here (!)
* JavaScript functionals are built to access components in the playwright page, and annotated page are returned as a result
* The intermediate "blinking effect" is the result of the "markPage()" function

In [1]:

import asyncio
from playwright.async_api import async_playwright
import base64
import time

with open("./src/mark_page.js") as f:
    mark_page_script = f.read()

async def mark_page(page):
    """ 
    This function marks the browser page and returns Image & BBoxes
    Useful -- what type is "page" here?
    """
    await page.evaluate(mark_page_script)
    for _ in range(10):
        try:
            bboxes = await page.evaluate("markPage()")
            break
        except:
            # May be loading...
            asyncio.sleep(3)

    time.sleep(4) # Sleeper here extends the period of us witnessing the annotation, which is in the stored image
    
    screenshot = await page.screenshot() # ScreenShot of the annotated page right here

    await page.evaluate("unmarkPage()") # This function remove the marking from the page

    return {
        "img": base64.b64encode(screenshot).decode(),
        "bboxes": bboxes,
    }

async def annotate(state):
    marked_page = await mark_page.with_retry().ainvoke(state["page"])
    return {**state, **marked_page}


# Need the surfer agent to help me insert this information to login-into linkedin page
my_linkedin_username = "fangyuan.yu18@gmail.com"
my_linkedin_password = "5826318"

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        # url = "https://www.google.com"
        url = "https://www.linkedin.com/in/matthewgeorgejohnson/recent-activity/all/"
        await page.goto(url)

        result_dict = await mark_page(page)
        bbox = result_dict["bboxes"]
        print("Bounding Boxes: ")
        print(bbox)

        # Annotate bboxes on Page
        annotate(result_dict)
        print("Annotation function pass through")

        import time
        time.sleep(88)
        # await browser.close()

        

# Since this is a Jupyter notebook, we should avoid using asyncio.run() directly in the main block.
# Instead, we can use the following approach to run the async function in notebooks:
import nest_asyncio
nest_asyncio.apply()

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(run())


Bounding Boxes: 
[{'x': 111, 'y': 46, 'type': 'svg', 'text': '', 'ariaLabel': ''}, {'x': 635, 'y': 203.9921875, 'type': 'input', 'text': '', 'ariaLabel': 'Email or phone'}, {'x': 635, 'y': 279.9921875, 'type': 'input', 'text': '', 'ariaLabel': 'Password'}, {'x': 756.45703125, 'y': 279.9921875, 'type': 'span', 'text': 'Show', 'ariaLabel': ''}, {'x': 550.875, 'y': 329.9921875, 'type': 'a', 'text': 'Forgot password?', 'ariaLabel': ''}, {'x': 635, 'y': 387.9921875, 'type': 'button', 'text': 'Sign in', 'ariaLabel': 'Sign in'}, {'x': 752.859375, 'y': 466.984375, 'type': 'a', 'text': 'User Agreement', 'ariaLabel': ''}, {'x': 515.484375, 'y': 484.984375, 'type': 'a', 'text': 'User Agreement', 'ariaLabel': ''}, {'x': 596.1328125, 'y': 484.984375, 'type': 'a', 'text': 'Privacy Policy', 'ariaLabel': ''}, {'x': 708.46875, 'y': 484.984375, 'type': 'a', 'text': 'Cookie Policy', 'ariaLabel': ''}, {'x': 560.5537719726562, 'y': 521.6237487792969, 'type': 'path', 'text': '', 'ariaLabel': ''}, {'x': 554.

/var/folders/bt/rngwf7wj73x2zfr_7chtd0jh0000gn/T/ipykernel_89590/3855757169.py:53: RuntimeWarning: coroutine 'annotate' was never awaited
  annotate(result_dict)


: 

: 